In [1]:
import os
import json5
from  tqdm import tqdm
import boto3
import yaml
from botocore.exceptions import NoCredentialsError, ClientError

In [2]:
def load_config(config_file):
    # Load configuration from the YAML file
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
    return config

In [3]:
def upload_file_to_s3(config, file_path, target_region_name, target_bucket_name, target_path, log_file='upload_file'):
    
    # Extract parameters from the configuration    
    aws_access_key_id = config['aws']['aws_access_key_id']
    aws_secret_access_key = config['aws']['aws_secret_access_key']
    log_file = log_file+'.csv'
    
    # Create an S3 client with the specified credentials
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=target_region_name
    )

    try:
        # Upload the file
        response = s3_client.upload_file(file_path, target_bucket_name, target_path + file_path.split('/')[-1])
        print(f"File {file_path} uploaded to {target_bucket_name}/{target_path}")
    except FileNotFoundError:
        print("The file was not found")
    except NoCredentialsError:
        print("Credentials not available")
    except ClientError as e:
        print(f"Failed to upload {file_path} to {target_bucket_name}/{target_path}: {e}")

In [4]:
def main():
    """s
    Upload single local file - Main entry point
    """    
    print("Current working directory:", os.getcwd())
    
    try:
        # Extract parameters from the configuration
        config_file = 'config/config.yaml'
        config = load_config(config_file)        

        # Source Folder and sub-folders to upload
        sources_folder_name = config['local']['sources_folder_name']

        file_name = 'lior_test.txt'
        file_path = f"{sources_folder_name}//{file_name}"
               
        # Target aws s3
        target_region_name = config['aws']['region_name']
        target_bucket_name = config['aws']['sources_bucket_name']
        target_path = config['aws']['sources_path']    
        log_file = 'upload_file'
        
        upload_file_to_s3(config, file_path, target_region_name, target_bucket_name, target_path, log_file)
        
    except Exception as e:
        print(f"Main process error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Current working directory: C:\github_repos\BIU_LLM_Project
File L:/My Drive/Source_Files//lior_test.txt uploaded to ct-external-sources/ds/rag01/sources/
